In [ ]:
from garminconnect import (
    Garmin,
    GarminConnectConnectionError,
    GarminConnectTooManyRequestsError,
    GarminConnectAuthenticationError,
)
import requests
from datetime import date
import numpy as np
import matplotlib.pyplot as plt
import cv2 # used for loading and processing images
import xmltodict
import os

today = date.today()

### Read your user name and password from a text file

In [ ]:
USER_EMAIL, USER_PASSWORD = np.loadtxt("./garmin_user_pass.txt",dtype=str)

### Login Script

In [ ]:
try:
    client = Garmin(USER_EMAIL, USER_PASSWORD)
    client.login()
    print(f"logged into Garmin as",client.get_full_name())

except (
    GarminConnectConnectionError,
    GarminConnectAuthenticationError,
    GarminConnectTooManyRequestsError,
) as err:
    print("Error occurred during Garmin Connect Client init: %s" % err)

except Exception as e:  # pylint: disable=broad-except
    print("Unknown error occurred during Garmin Connect Client init")
    print(e)

### Get Your N Most Recent Activities

In [ ]:
N_ACTIVITIES = 3

activities = client.get_activities(0,N_ACTIVITIES) # 0=start, 1=limit
activities_data = []
for act in activities:
    activity_id = act["activityId"] # Get the id
    gpx_data = client.download_activity(activity_id, dl_fmt=client.ActivityDownloadFormat.GPX)
    activities_data.append(gpx_data)
    
    #Write the file to disk
    output_file = f"./{str(activity_id)}.gpx"
    with open(output_file, "wb") as fb:
        fb.write(gpx_data)

### Object to handle sampling green space data

In [68]:
class image_sampler:
    
    def __init__(self, image_path, extent):
        #This is handled very badly
        if(os.path.exists(image_path)):
            print("loading image")
            self.image = cv2.imread(image_path, -1)
            self.image_shape = self.image.shape
            self.extent = extent
        else:
            print("bad image path")
        
        
    def sample_image(self, point):
        #pixel_ref = point_to_pixelref(image.shape, extent, sample_point)
        dx = self.extent[0][0] - self.extent[1][0]
        dy = self.extent[0][1] - self.extent[1][1]

        new_x = point[0] - self.extent[1][0]
        new_y = point[1] - self.extent[1][1]

        pc_x = new_x / dx
        pc_y = new_y / dy

        x = self.image_shape[0] - round(self.image_shape[0] * pc_x)
        y = self.image_shape[1] - round(self.image_shape[1] * pc_y)
        
        return (self.image[x, y] > 0)
    
    def sample_activity(self, activity):
        if(activity['gpx']['trk']['trkseg'] == None): # all activities seem to have these attributes?
            print("invalid activity")
            return False
        trkpts = activity['gpx']['trk']['trkseg']['trkpt']
        flag = False
        for point in trkpts:
            if(self.sample_image([float(point['@lat']), float(point['@lon'])])):
                flag = True
                break
        return flag

### Instance the object with some data

In [72]:
extent = [[60.810092954, -9.140209406], [49.847346159, 2.679817307]]  # wgs84 extent of UK
green_space_data_path = "./merged_greenspaces.png"
sampler = image_sampler(green_space_data_path, extent)

loading image


### Lets sample some dummy locations

In [73]:
location0 = [51.581207,-0.150764] # highgate wood
location1 = [51.564395,-0.163432] # hampstead heath
location2 = [51.578052,-0.205430] # somewhere not green

locations = [location0, location1, location2]

for l in locations:
    print(sampler.sample_image(l))

True
True
False


### Lets now sample an activity and see if it went into a green space

In [74]:
activity = activities_data[0]
for i,a in enumerate(activities_data):
    print(f"Is activity {i} in a green space?", sampler.sample_activity(xmltodict.parse(a)))

invalid activity
Is activity 0 in a green space? False
invalid activity
Is activity 1 in a green space? False
Is activity 2 in a green space? True
